# Chapter 10. Sequence Hacking, Hashing, and Slicing
- Basic Sequence protocol: `__len__` and `__getitem__`.
- Safe representation of instances with many items.
- Proper slicing support, producing new Vector instances.
- Aggregate hashing taking into account every contained element value.
- Custom formatting language extension.

# Vector: A User-Defined Sequence Type

In [22]:
from array import array
import reprlib
import math
import functools
import operator

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
    
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        # Below code is inefficient for large length instances
        # return tuple(self) == tuple(other)
        
        # Efficient example 1
        # if len(self) != len(other):
        #     return False
        # for a, b in zip(self, other):
        #     if a != b:
        #         return False
        # return True
        
        # Efficinet example 2
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
        
    
    def __hash__(self):
        # hashes = (hash(x) for x in self._components)
        hashes = map(hash, self._components)
        # map method is lazy because it creates a generator to save memory.
        # This is the same as genexp
        return functools.reduce(operator.xor, hashes, 0)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    shortcut_names = 'xyzt'
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self.components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
    
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n - 1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a
    
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))
    
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
        


## Vector Take #1: Vector2d Compatible

## Vector Take #2: A Sliceable Sequence
- `__len__`
- `__getitem__`

### How Slicing Works

In [1]:
class MySeq:
    def __getitem__(self, index):
        return index

In [2]:
s = MySeq()
s[1]

1

In [3]:
s[1:4]

slice(1, 4, None)

In [4]:
s[1:4:2]

slice(1, 4, 2)

In [5]:
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [6]:
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

In [7]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [8]:
slice(None, 10, 2).indices(5)

(0, 5, 2)

In [9]:
slice(-3, None, None).indices(5)

(2, 5, 1)

## Vector Take #3: Dynamimc Attribute Access
- The `__getattr__` method is invoked by the interpreter when attribute lookup fails.
 - e.g. my_obj.x
     - check my_obj instance has an attribute named x
     - search the inheritance graph of `my_obj.__class__` for x attribute 
     - call `__getattr__` defined in `my_obj` with `self` and `x` as `str`
- `__getatrr__` is called only when the instance doesn't have the attribute. Once invoked, the instance hold the attribute, so `__getattr__` is never called in relation to the instance.

In [16]:
lst = [range(3)]
lst

[range(0, 3)]

In [17]:
type(lst)

list

In [18]:
type(type(lst))

type

In [21]:
dir(type(lst))

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

## Vector Take #4: Hashing and a Faster ==
- `__eq__`
- `__hash__`

<br>

- `functools.reduce(function, iterable, initializer)`
    - should pass initializer for when iterable length is 0
    - initializer should be
        - 0 for +, |, ^
        - 1 for *, &

In [23]:
from itertools import zip_longest
list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3], fillvalue=-1))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (-1, -1, 3.3)]

## Vector Take #5: Formatting